In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose


In [3]:
def draw_landmarks(frame, landmarks):
    fh, fw, _ = frame.shape
    
    FONT_SIZE = 25
    FONT_COLOR = "#CD5567"
    FONT_FAMILY = cv2.FONT_HERSHEY_COMPLEX
    FONT_THICKNESS = 2
    FONT_SCALE = 1.2
    
    for landmark_code, landmark in enumerate(landmarks):
        x, y, z = landmark.x, landmark.y, landmark.z
        if z!=0:
            x0, y0 = int(x/abs(z)*fw), int(y/abs(z)/fh)
        else:
            x0, y0 = int(x*fw), int(y*fh)
            
        cv2.putText(frame, str(landmark_code), (x0, y0), FONT_FAMILY, FONT_SCALE, FONT_COLOR, FONT_THICKNESS)
        
        cv2.circle(frame, (x0, y0), 5, (255, 0, 0), -1)

In [6]:
def calculate_angle(pos1, pos2, pos3):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    angle_bw = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(b[1]-a[1], b[0]-a[0])
    
    angle_bw *= int(180.0/np.pi)
    if angle_bw > 180.0:
        angle_bw -= 180.0
        
    return angle_bw

In [7]:
def process(frames, frame_len, rgb_frames):
    state = "up" # Values are "push" and "up"
    pushup_counter = 0
    
    refined_frames = []

    for frame_idx in range(frame_len):
        frame = frames[frame_idx].copy()
        rgb_frame = rgb_frames[frame_idx].copy()

        results = pose_detection.process(rgb_frame)
        
        if results.pose_landmarks:
            drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            landmarks = results.pose_landmarks.landmark
            
            if (landmarks[25].y - landmarks[11].y) < 0.5:
                angle_for_pushups = calculate_angle(landmarks[11], landmarks[13], landmarks[15])
                
                if state=="up" and angle_for_pushups < 90:
                    state = "push"
                
                if state == "push" and angle_for_pushups > 160:
                    state = "up"
                    pushup_counter += 1
                    print(pushup_counter, frame_idx)
            
        # Text settings
        frame_height, frame_width, _ = frame.shape
        if pushup_counter % 2:
            position = int(frame_width*.5), int(frame_height*.74)
        else:
            position = int(frame_width*.5), int(frame_height*.75)
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.2
        color = (0, 255, 0)           # BGR -> Green
        thickness = 2
        line_type = cv2.LINE_AA
        
        text = f"{pushup_counter}"
        cv2.putText(frame, text, position, font, font_scale, color, thickness, line_type)
        refined_frames.append(frame)
        cv2.imshow("Frame", frame)
            
        key = cv2.waitKey(1)
        if key == ord("q"):
            break

    cv2.destroyWindow("Frame")
    return refined_frames

In [10]:
def scale_the_frame(frame, scaling_factor=0.25):
    fh, fw, _ = frame.shape
    ow, oh = int(fw*scaling_factor), int(fh*scaling_factor)
    image = cv2.resize(frame, (ow, oh))
    return image

In [8]:
def get_frames(VIDEO_PATH, scaling_factor=0.25):
    vid = cv2.VideoCapture(VIDEO_PATH)
    frames = []
    while True:
        ret, frame = vid.read()
        if not ret:
            break
        
        fh, fw, _ = frame.shape # as it is most closest to an Numpy Array
        frame = scale_the_frame(frame, scaling_factor=scaling_factor)
        frames.append(frame)

    vid.release()
    cv2.destroyAllWindows()

    return len(frames), frames

In [11]:
PATH_TO_VIDEO = os.path.join("pushup_vids", "Day_46.mp4")
pose_detection = mp_pose.Pose()
frame_len, frames = get_frames(PATH_TO_VIDEO)

In [12]:
frame_len

2057